In [ ]:
!pip install -q qiskit qiskit-nature qiskit-aer pyscf qiskit_algorithms

In [ ]:
# pylint: disable=line-too-long
import qiskit_nature
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_algorithms.optimizers import SLSQP,SPSA
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

qiskit_nature.settings.use_pauli_sum_op = False  # pylint: disable=undefined-variable
# pylint: enable=line-too-long
from qiskit_nature.second_q.drivers import PySCFDriver
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2
import numpy as np
import pyscf


from qiskit_nature.second_q.algorithms import ExcitedStatesEigensolver
from qiskit_algorithms import NumPyEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, QEOM, EvaluationRule
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper

In [ ]:
from qiskit.primitives import Estimator



molecule = MoleculeInfo(
symbols=["C", "C", "C", "C", "C", "C", "S", "O", "C", "O", "C", "C", "C", "C", "C", "N", "C", "C", "C", "C", "C", "C", "C", "C", "C", "C", "C", "C", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H"],
coords=(
    [-0.2316640, 1.1348450, 0.6956120],
    [-0.8886300, 0.3253780, -0.2344140],
    [-0.1842470, -0.1935670, -1.3239330],
    [1.1662930, 0.0801450, -1.4737160],
    [1.8089230, 0.8832220, -0.5383540],
    [1.1155860, 1.4218050, 0.5392780],
    [3.5450920, 1.2449890, -0.7349240],
    [3.8606900, 1.0881590, -2.1541690],
    [4.3889120, -0.0620730, 0.1436780],
    [3.8088290, 2.4916780, -0.0174650],
    [4.6830900, 0.1064460, 1.4918230],
    [5.3364470, -0.9144080, 2.1705280],
    [5.6895490, -2.0818670, 1.5007820],
    [5.4000540, -2.2323130, 0.1481350],
    [4.7467230, -1.2180160, -0.5404770],
    [-2.2589180, 0.0399120, -0.0793330],
    [-2.8394600, -1.2343990, -0.1494160],
    [-4.2635450, -1.0769890, 0.0660760],
    [-4.5212550, 0.2638010, 0.2662190],
    [-3.2669630, 0.9823890, 0.1722720],
    [-2.2678900, -2.4598950, -0.3287380],
    [-3.1299420, -3.6058560, -0.3236210],
    [-4.5179520, -3.4797390, -0.1395160],
    [-5.1056310, -2.2512990, 0.0536940],
    [-5.7352450, 1.0074800, 0.5140960],
    [-5.6563790, 2.3761270, 0.6274610],
    [-4.4287740, 3.0501460, 0.5083650],
    [-3.2040560, 2.3409470, 0.2746950],
    [-0.7813570, 1.5286610, 1.5426490],
    [-0.7079140, -0.7911480, -2.0611600],
    [1.7161320, -0.2933710, -2.3302930],
    [1.6308220, 2.0660550, 1.2427990],
    [4.4214900, 1.0345500, 1.9875450],
    [5.5773000, -0.7951290, 3.2218590],
    [6.2017810, -2.8762260, 2.0345740],
    [5.6906680, -3.1381740, -0.3739110],
    [4.5337010, -1.3031330, -1.6001680],
    [-1.1998460, -2.5827750, -0.4596910],
    [-2.6937370, -4.5881470, -0.4657540],
    [-5.1332290, -4.3740010, -0.1501080],
    [-6.1752900, -2.1516170, 0.1987120],
    [-6.6812260, 0.4853900, 0.6017680],
    [-6.5574610, 2.9529350, 0.8109620],
    [-4.3980410, 4.1305040, 0.5929440],
    [-2.2726630, 2.8838620, 0.1712760]),
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

driver = PySCFDriver.from_molecule(molecule)
driver.max_memory=3000


properties = driver.run()





In [ ]:
# Now you can get the reduced electronic structure problem
problem =  ActiveSpaceTransformer(num_electrons=2, num_spatial_orbitals=2).transform(properties)

num_particles = problem.num_particles
num_spatial_orbitals = problem.num_spatial_orbitals


mapper = ParityMapper()

In [ ]:
optimizer = SLSQP(maxiter=10)
noiseless_estimator = Estimator()


exact_energies=[]

qeomvqe_energies=[]

def filter_criterion(eigenstate, eigenvalue, aux_values):
    return np.isclose(aux_values["ParticleNumber"][0], 2.0) and np.isclose(
        aux_values["Magnetization"][0], 0.0
    )






numpy_solver = NumPyEigensolver(k=4, filter_criterion=filter_criterion)

numpy_excited_states_solver = ExcitedStatesEigensolver(mapper, numpy_solver)
numpy_results = numpy_excited_states_solver.solve(problem)

print(numpy_results)

init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
var_form = UCCSD(
    num_spatial_orbitals, num_particles, mapper, initial_state=init_state
)
vqe = VQE(
    noiseless_estimator,
    var_form,
    optimizer,
    initial_point=[0.0] * var_form.num_parameters,
)

gse = GroundStateEigensolver(mapper, vqe)

qeom_excited_states_solver = QEOM(gse, noiseless_estimator, "sd", EvaluationRule.ALL)

qeom_results = qeom_excited_states_solver.solve(problem)


exact_energies.append(numpy_results)
qeomvqe_energies.append(qeom_results)

print("All energies have been calculated")

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -4025.092863300766
  - computed part:      -0.662405554192
  - ActiveSpaceTransformer extracted energy part: -4024.430457746574
~ Nuclear repulsion energy (Hartree): 2523.279371819741
> Total ground state energy (Hartree): -1501.813491481025
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -4024.830753216854
> Total excited state energy (Hartree): -1501.551381397113
  2: 
* Electronic excited state energy (Hartree): -4024.652451404058
> Total excited state energy (Hartree): -1501.373079584317
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  1:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  2:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-0.00001512  0.0  -0.0000189]
 
  0: 
  * Electronic dipole moment (a.u.): [0.464630049045  1.408542996334  -0.93452215742]
    - comput

In [ ]:
print(qeom_results)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -4025.092863289239
  - computed part:      -0.662405542665
  - ActiveSpaceTransformer extracted energy part: -4024.430457746574
~ Nuclear repulsion energy (Hartree): 2523.279371819741
> Total ground state energy (Hartree): -1501.813491469498
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -4024.924201059717
> Total excited state energy (Hartree): -1501.644829239976
  2: 
* Electronic excited state energy (Hartree): -4024.830742303155
> Total excited state energy (Hartree): -1501.551370483414
  3: 
* Electronic excited state energy (Hartree): -4024.65244010903
> Total excited state energy (Hartree): -1501.373068289289
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  1:  # Particles: 2.000 S: 1.000 S^2: 2.000 M: -0.000
  2:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000
  3:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE